# 실습 세션 3: 여행 플래너 Multi-Agent 앱의 Docker 빌드 및 AKS 배포

이 세션에서는 세션 2에서 만든 여행 플래너 멀티에이전트 앱을 Docker 이미지로 빌드하고, Azure Kubernetes Service(AKS)에 배포하는 핵심 실습만 다룹니다.

## 1. Docker CLI 설치 확인 (GitHub Codespaces 환경)

GitHub Codespaces 환경에는 이미 Docker CLI가 설치되어 있으므로 별도의 설치가 필요하지 않습니다.

- Codespaces 터미널에서 아래 명령어로 Docker CLI가 정상적으로 설치되어 있는지 확인하세요.

```sh
docker --version
```

- 위 명령어 실행 시 Docker 버전 정보가 출력되면 정상입니다.
- 로컬 환경에서 실습할 경우 [Docker Desktop](https://www.docker.com/products/docker-desktop/)을 별도로 설치해야 합니다.

## 2. Dockerfile 작성

아래는 여행 플래너 앱을 컨테이너화하기 위한 Dockerfile 예시입니다.

In [ ]:
%%writefile Dockerfile
# Step 1: Base image with Python environment
FROM python:3.12-slim AS base

# Step 2: Set working directory
WORKDIR /app

# Step 3: Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Step 4: Copy the project files into the container
COPY . .

# Step 5: Define the entry point
ENTRYPOINT ["python", "/app/travel_planning_agent.py"]

## 2. Docker 이미지 빌드

아래 명령어로 Docker 이미지를 빌드합니다. (로컬에서 실행)

In [ ]:
# Docker 이미지 빌드
!docker build -t travel-planning-agent-app:v1 .

## 3. Azure CLI 설치 (GitHub Codespaces 환경)

GitHub Codespaces 환경에는 기본적으로 Azure CLI(az)가 설치되어 있지 않을 수 있습니다. 아래 명령어로 Azure CLI를 설치하세요.
```
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
```

- 설치 후, `az --version` 명령어로 정상 설치 여부를 확인하세요.
- 이후 단계에서 az 명령어를 사용해 Azure 리소스를 생성할 수 있습니다.

## 4. AKS 클러스터 및 ACR 레지스트리 생성

아래 명령어로 Azure에서 컨테이너 레지스트리(ACR)와 AKS 클러스터을 생성합니다.

먼저 Azure 계정에 로그인하세요:

In [ ]:
# Azure 계정 로그인
!az login

In [ ]:
# Azure 리소스 그룹 생성 (예시)
# !az group create --name myResourceGroup --location koreacentral

# Azure Container Registry 생성 (예시)
# !az acr create --resource-group myResourceGroup --name <ACR-REGISTRY-NAME> --sku Basic

# AKS 클러스터 생성 (예시)
# !az aks create --resource-group myResourceGroup --name myAKSCluster --node-count 1 --enable-addons monitoring --generate-ssh-keys --attach-acr <ACR-REGISTRY-NAME>

# AKS 클러스터 인증 정보 가져오기
# !az aks get-credentials --resource-group myResourceGroup --name myAKSCluster

이제 위에서 생성한 ACR과 AKS를 활용해 이미지를 푸시하고, 클러스터에 배포할 수 있습니다.

## 5. Kubernetes 배포를 위한 YAML 파일 작성

아래는 AKS에 배포할 때 사용할 수 있는 예시 deployment.yaml 입니다.

In [ ]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: travel-planning-agent-app
spec:
  replicas: 1
  selector:
    matchLabels:
      app: travel-planning-agent-app
  template:
    metadata:
      labels:
        app: travel-planning-agent-app
    spec:
      containers:
      - name: travel-planning-agent-app
        image: <ACR-REGISTRY>/travel-planning-agent-app:v1  # 실제 ACR 경로로 교체

## 6. AKS에 배포 (kubectl)

아래 명령어로 이미지를 ACR에 푸시하고, AKS에 배포합니다.

In [ ]:
# 1. ACR에 이미지 푸시 (예시)
# !az acr login --name <ACR-REGISTRY-NAME>
# !docker tag travel-planning-agent-app:v1 <ACR-REGISTRY>/travel-planning-agent-app:v1
# !docker push <ACR-REGISTRY>/travel-planning-agent-app:v1

# 2. AKS에 배포
# !kubectl apply -f deployment.yaml

---

- `<ACR-REGISTRY>`는 실제 Azure Container Registry 주소로 교체하세요.
- 필요시 replicas, 리소스, 네트워크 설정 등은 YAML에서 조정하세요.

## 7. Log Analytics Workspace에서 컨테이너 로그 조회

AKS 클러스터를 생성할 때 Log Analytics Workspace를 연동했다면, 아래 방법으로 컨테이너 로그를 확인할 수 있습니다.

- Azure Portal에서 AKS 클러스터 > 모니터링 > 로그(Logs) 메뉴로 이동
- 아래와 같은 Kusto 쿼리로 컨테이너 로그를 조회할 수 있습니다.

In [ ]:
# 예시: 컨테이너 v2 로그 Kusto 쿼리
# ContainerLogV2
# | where ContainerName == "travel-planning-agent-app"
# | sort by TimeGenerated desc
# | take 100

- 필요에 따라 PodName, Namespace, LogLevel 등으로 필터링할 수 있습니다.
- 실시간 로그 스트림이 필요하다면 Azure Portal의 "실시간 로그" 기능을 활용하거나, kubectl logs 명령어도 사용할 수 있습니다.

## 참고 자료 및 공식 가이드

- [Azure Kubernetes Service(AKS) 공식 문서](https://learn.microsoft.com/azure/aks/)
- [Azure Container Registry(ACR) 공식 문서](https://learn.microsoft.com/azure/container-registry/)
- [Docker 공식 문서](https://docs.docker.com/)
- [Kubernetes 공식 문서](https://kubernetes.io/docs/)
- [Azure CLI 공식 문서](https://learn.microsoft.com/cli/azure/)

위 링크를 참고하여 각 기술의 상세 사용법과 추가 예제를 확인할 수 있습니다.